
PONTIFICIA UNIVERSIDAD JAVERIANA

Autor: Luis Montenegro

Fecha: 22/07/2024

Tema: Introducción al procesamiento de altos volúmenes de datos.

In [0]:
###Se importan las bibliotecas necesarias.
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, FloatType

In [0]:
####Databricks es un entorno que proporciona cómputo y almacenamiento.
###Community Edition permite acceder a parte de estos recursos de forma gratuita.

### Se observan los datasets (conjuntos de datos) disponibles en DataBricks.
display(dbutils.fs.ls("databricks-datasets/samples"))

path,name,size,modificationTime
dbfs:/databricks-datasets/samples/adam/,adam/,0,0
dbfs:/databricks-datasets/samples/data/,data/,0,0
dbfs:/databricks-datasets/samples/docs/,docs/,0,0
dbfs:/databricks-datasets/samples/lending_club/,lending_club/,0,0
dbfs:/databricks-datasets/samples/newsgroups/,newsgroups/,0,0
dbfs:/databricks-datasets/samples/people/,people/,0,0
dbfs:/databricks-datasets/samples/population-vs-price/,population-vs-price/,0,0


In [0]:
####Como ejemplo, se carga un fichero.
###La carga se instancia en un objeto DATAFRAME, de Spark.
dfejemplo00 = spark.read.csv("/databricks-datasets/samples/population-vs-price/data_geo.csv", header = True)
###Se presenta el dataframe.
dfejemplo00.show(4)

+---------+----------+-------+----------+------------------------+-----------------------+
|2014 rank|      City|  State|State Code|2014 Population estimate|2015 median sales price|
+---------+----------+-------+----------+------------------------+-----------------------+
|      101|Birmingham|Alabama|        AL|                  212247|                  162.9|
|      125|Huntsville|Alabama|        AL|                  188226|                  157.7|
|      122|    Mobile|Alabama|        AL|                  194675|                  122.5|
|      114|Montgomery|Alabama|        AL|                  200481|                    129|
+---------+----------+-------+----------+------------------------+-----------------------+
only showing top 4 rows



In [0]:
###Se impriment los nombres de las columnas.
dfejemplo00.columns

['2014 rank',
 'City',
 'State',
 'State Code',
 '2014 Population estimate',
 '2015 median sales price']

In [0]:
###Se presenta un resumen estadístico.
dfejemplo00.describe().show()

+-------+-----------------+-------+---------+----------+------------------------+-----------------------+
|summary|        2014 rank|   City|    State|State Code|2014 Population estimate|2015 median sales price|
+-------+-----------------+-------+---------+----------+------------------------+-----------------------+
|  count|              294|    294|      294|       294|                     293|                    109|
|   mean|            147.5|   NULL|     NULL|      NULL|      307284.89761092153|      211.2660550458717|
| stddev|85.01470461043782|   NULL|     NULL|      NULL|       603487.8272175139|       134.017245449277|
|    min|                1|Abilene|  Alabama|        AK|                  101408|                  105.2|
|    max|               99|Yonkers|Wisconsin|        WI|                  912791|                   95.4|
+-------+-----------------+-------+---------+----------+------------------------+-----------------------+



In [0]:
####Comando para presentar esquema.
dfejemplo00.printSchema()

root
 |-- 2014 rank: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- 2014 Population estimate: string (nullable = true)
 |-- 2015 median sales price: string (nullable = true)



In [0]:
###Se recomienda cambiar los nombres de las columnas.
dfejemplo01 = dfejemplo00.withColumnRenamed("2014 rank","2014_rank")\
    .withColumnRenamed("State Code","State_Code")\
    .withColumnRenamed("2014 Population estimate","2014_Population_estimate")\
    .withColumnRenamed("2015 median sales price","2015_median_sales_price")
display(dfejemplo01)


###Observaciones con los tipos de datos:
  -Se observa que todos los datos son de tipo STRING, incluso aquellos que debieran ser de tipo numérico (INTEGER o FLOAT).

  -Se deben ajustar los datos a su tipo correspondiente.
  
  -Para ello, se usará como herramienta el método ***.withColumn()*** (cambio de tipo manual).

In [0]:
###Se hace el casteo o modificación de cada tipo de dato
dfejemplo02 = dfejemplo01.withColumn("2014_rank", col("2014_rank").cast(IntegerType()))\
    .withColumn("2014_Population_estimate", col("2014_Population_estimate").cast(IntegerType()))\
    .withColumn("2015_median_sales_price", col("2015_median_sales_price").cast(FloatType()))
###Se hace inpescción de la reciente modificación de datos.
dfejemplo02.printSchema()

root
 |-- 2014_rank: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- State_Code: string (nullable = true)
 |-- 2014_Population_estimate: integer (nullable = true)
 |-- 2015_median_sales_price: float (nullable = true)



In [0]:
###Nuevamente se revisa el resumen estadístico de los datos.
display(dfejemplo02.describe())

summary,2014_rank,City,State,State_Code,2014_Population_estimate,2015_median_sales_price
count,294,294,294,294,293,109
mean,147.5,null,null,null,307284.89761092153,211.26605476589378
stddev,85.01470461043782,null,null,null,603487.8272175139,134.01724508539056
min,1,Abilene,Alabama,AK,101408,78.6
max,294,Yonkers,Wisconsin,WI,8491079,900.0



###Vista de Datos

El comando ***createOrReplaceTempView*** crea una vista temporal usando un SparkDataFrame en la sesión ***spark***. Esto último nos permite hacer consultas ***SQL*** sobre el objeto.

In [0]:
###Se crea la vista temporal.
dfejemplo02.createOrReplaceTempView("data_geo")

Se pueden elaborar consultas SQL de manera rápida usando ***%sql***.

In [0]:
###Se pueden hacer consultas SQL sobre el objeto de vista temporal.
%sql

select `State_Code`, `2015_median_sales_price` from data_geo

  File <command-1991233493054403>, line 4
    select `State_Code`, `2015_median_sales_price` from data_geo
                              ^
SyntaxError: invalid decimal literal
